In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [ ]:
# importer des datas
data = pd.read_csv('data.csv')

In [ ]:
# choisir aléatoirement 1000 échantillons de chaque type comme training datas, 200 comme testing data
train_data = data.groupby('category').sample(n=1000, random_state=42)
test_data = data.groupby('category').sample(n=200, random_state=42)

In [ ]:
# caractéristique et label
X_train, y_train = train_data['description'], train_data['category']
X_test, y_test = test_data['description'], test_data['category']

In [ ]:
# encodgae de label
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
# vectorizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=200)

In [ ]:
# paramètres de modèle
vocab_size = len(tokenizer.word_index) + 1  # taille de vocabulaire
embedding_dim = 128  # embedding layer
lstm_units = 128  # nombre de "units" de LSTM

In [ ]:
# construction de modèle
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=200))
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()
# des paramètres
batch_size = 64
epochs = 10

In [ ]:
# enrtraînement
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

In [ ]:
# prédiction
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)

In [ ]:
# calculer
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
print(report)

In [ ]:
# matrix
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.rcParams['font.sans-serif']=['SimHei'] #affichier des étiquettes en chinois
plt.ylabel('catégorie réel')
plt.xlabel('catégorie prédite')
plt.show()